# LSTM Stock Predictor Using Closing Prices

In this notebook, you will build and train a custom LSTM RNN that uses a 10 day window of Bitcoin closing prices to predict the 11th day closing price. 

You will need to:

1. Prepare the data for training and testing
2. Build and train a custom LSTM RNN
3. Evaluate the performance of the model

## Data Preparation

In this section, you will need to prepare the training and testing data for the model. The model will use a rolling 10 day window to predict the 11th day closing price.

You will need to:
1. Use the `window_data` function to generate the X and y values for the model.
2. Split the data into 70% training and 30% testing
3. Apply the MinMaxScaler to the X and y values
4. Reshape the X_train and X_test data for the model. Note: The required input format for the LSTM is:

```python
reshape((X_train.shape[0], X_train.shape[1], 1))
```

In [84]:
import numpy as np
import pandas as pd
import hvplot.pandas

In [85]:
# Set the random seed for reproducibility
# Note: This is for the prototyping, comment this out and run multiple experiments to evaluate your model
from numpy.random import seed
seed(1)
from tensorflow import random
random.set_seed(2)

In [86]:
# Load the fear and greed sentiment data for Bitcoin
df = pd.read_csv('btc_sentiment.csv', index_col="date", infer_datetime_format=True, parse_dates=True)
df = df.drop(columns="fng_classification")
df.head()

,fng_value
date,
2019-07-29,19
2019-07-28,16
2019-07-27,47
2019-07-26,24
2019-07-25,42


In [87]:
# Load the historical closing prices for Bitcoin
df2 = pd.read_csv('btc_historic.csv', index_col="Date", infer_datetime_format=True, parse_dates=True)['Close']
df2 = df2.sort_index()
df2.tail()

Date
2019-07-25    9882.429688
2019-07-26    9847.450195
2019-07-27    9478.320313
2019-07-28    9531.769531
2019-07-29    9529.889648
Name: Close, dtype: float64

In [88]:
# Join the data into a single DataFrame
df = df.join(df2, how="inner")
df.tail()

,fng_value,Close
2019-07-25,42,9882.429688
2019-07-26,24,9847.450195
2019-07-27,47,9478.320313
2019-07-28,16,9531.769531
2019-07-29,19,9529.889648


In [89]:
df.head()

,fng_value,Close
2018-02-01,30,9114.719727
2018-02-02,15,8870.820313
2018-02-03,40,9251.269531
2018-02-04,24,8218.049805
2018-02-05,11,6937.080078


In [90]:
# This function accepts the column number for the features (X) and the target (y)
# It chunks the data up with a rolling window of Xt-n to predict Xt
# It returns a numpy array of X any y
def window_data(df, window, feature_col_number, target_col_number):
    X = []
    y = []
    for i in range(len(df) - window - 1):
        features = df.iloc[i:(i + window), feature_col_number]
        target = df.iloc[(i + window), target_col_number]
        X.append(features)
        y.append(target)
    return np.array(X), np.array(y).reshape(-1, 1)

In [91]:
# Predict Closing Prices using a 10 day window of previous closing prices
# Then, experiment with window sizes anywhere from 1 to 10 and see how the model performance changes
window_size = 10

# Column index 0 is the 'fng_value' column
# Column index 1 is the `Close` column
feature_column = 1
target_column = 1
X, y = window_data(df, window_size, feature_column, target_column)

In [92]:
y

array([[ 8084.609863],
       [ 8911.269531],
       [ 8544.69043 ],
       [ 9485.639648],
       [10033.75    ],
       [10188.730469],
       [11097.209961],
       [10417.230469],
       [11182.280273],
       [11256.429688],
       [10481.660156],
       [ 9847.959961],
       [10175.509766],
       [ 9705.730469],
       [ 9610.110352],
       [10326.5     ],
       [10594.759766],
       [10334.44043 ],
       [10929.370117],
       [11043.120117],
       [11465.360352],
       [11504.419922],
       [11440.730469],
       [10735.450195],
       [ 9928.55957 ],
       [ 9316.719727],
       [ 9252.759766],
       [ 8797.269531],
       [ 9544.839844],
       [ 9142.150391],
       [ 9160.120117],
       [ 8217.700195],
       [ 8268.410156],
       [ 8283.230469],
       [ 7883.450195],
       [ 8215.400391],
       [ 8623.139648],
       [ 8920.709961],
       [ 8911.349609],
       [ 8724.980469],
       [ 8934.799805],
       [ 8548.05957 ],
       [ 8472.839844],
       [ 81

In [93]:
# Use 70% of the data for training and the remainder for testing
def custom_split(X, y, perc_train):
    """custom train test split on np arrays"""
    where_to_split = (int(len(X)*perc_train))

    X_training = X[0:where_to_split]
    X_testing = X[where_to_split:]

    y_training = y[0:where_to_split]
    y_testing = y[where_to_split:]
    
    return X_training , X_testing , y_training , y_testing

In [94]:
splt_data = [d for d in custom_split(X=X, y=y, perc_train=.70)]

In [95]:
from sklearn.preprocessing import MinMaxScaler
def fit_transform_custom(train_test_lists, scale_method = 'MinMaxScaler'):
    """Takes split data and scales it with a Scaler Method from the sklearn.preprocessing library. 
    train_test_list must follow this formatting rule[X_train, X_test, y_train, y_test]"""
    X_train = train_test_lists[0]
    X_test = train_test_lists[1]

    y_train = train_test_lists[2]
    y_test = train_test_lists[3]

    if scale_method == 'MinMaxScaler':
        scaler = MinMaxScaler()
        # X fit-Transform
        scaler.fit(X_train)
        X_train = scaler.transform(X_train)
        X_test = scaler.transform(X_test)
        # y Fit-Transform
        scaler.fit(y_train)
        y_train = scaler.transform(y_train)
        y_test = scaler.transform(y_test)
        
    return X_train , X_test , y_train , y_test

In [96]:
scaled_data = [d for d in fit_transform_custom(splt_data)]

### Reshape Features Data for the LSTM Model

The LSTM API from Keras needs to receive the features data as a _vertical vector_, so that we need to reshape the `X` data in the form `reshape((X_train.shape[0], X_train.shape[1], 1))`.

Both sets, training, and testing are reshaped.

In [83]:
# Reshape the features for the model
def LSTM_reshape_features(data_list):
    """Reshape features data for LSTM. Features data must be the first two arrays passed to this method"""
    X_train = data_list[0]
    X_test = data_list[1]
    data_list[0] = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
    data_list[1]= X_test.reshape((X_test.shape[0], X_test.shape[1], 1))
    return None

In [99]:
LSTM_reshape_features(scaled_data)


---

## Build and Train the LSTM RNN

In this section, you will design a custom LSTM RNN and fit (train) it using the training data.

You will need to:
1. Define the model architecture
2. Compile the model
3. Fit the model to the training data

### Hints:
You will want to use the same model architecture and random seed for both notebooks. This is necessary to accurately compare the performance of the FNG model vs the closing price model. 

In [103]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [13]:
# Build the LSTM model. 
# The return sequences need to be set to True if you are adding additional LSTM layers, but 
# You don't have to do this for the final layer. 
# Note: The dropouts help prevent overfitting
# Note: The input shape is the number of time steps and the number of indicators
# Note: Batching inputs has a different input shape of Samples/TimeSteps/Features

# YOUR CODE HERE!


In [14]:
# Compile the model
# YOUR CODE HERE!

In [15]:
# Summarize the model
# YOUR CODE HERE!

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 10, 30)            3840      
_________________________________________________________________
dropout (Dropout)            (None, 10, 30)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 10, 30)            7320      
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 30)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 30)                7320      
_________________________________________________________________
dropout_2 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense (Dense)                (None, 1)                 3

In [16]:
# Train the model
# Use at least 10 epochs
# Do not shuffle the data
# Experiement with the batch size, but a smaller batch size is recommended
# YOUR CODE HERE!

Epoch 1/10
371/371 [==============================] - 2s 5ms/step - loss: 0.0276
Epoch 2/10
371/371 [==============================] - 2s 5ms/step - loss: 0.0241
Epoch 3/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0290
Epoch 4/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0282
Epoch 5/10
371/371 [==============================] - 2s 5ms/step - loss: 0.0246
Epoch 6/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0230
Epoch 7/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0226
Epoch 8/10
371/371 [==============================] - 2s 6ms/step - loss: 0.0198
Epoch 9/10
371/371 [==============================] - 2s 5ms/step - loss: 0.0199
Epoch 10/10
371/371 [==============================] - 2s 5ms/step - loss: 0.0184


---

## Model Performance

In this section, you will evaluate the model using the test data. 

You will need to:
1. Evaluate the model using the `X_test` and `y_test` data.
2. Use the X_test data to make predictions
3. Create a DataFrame of Real (y_test) vs predicted values. 
4. Plot the Real vs predicted values as a line chart

### Hints
Remember to apply the `inverse_transform` function to the predicted and y_test values to recover the actual closing prices.

In [17]:
# Evaluate the model
# YOUR CODE HERE!

5/5 [==============================] - 0s 2ms/step - loss: 0.0487


0.048721764236688614

In [18]:
# Make some predictions
# YOUR CODE HERE!

In [19]:
# Recover the original prices instead of the scaled version
predicted_prices = y_test_scaler.inverse_transform(predicted)
real_prices = y_test_scaler.inverse_transform(y_test.reshape(-1, 1))

In [20]:
# Create a DataFrame of Real and Predicted values
stocks = pd.DataFrame({
    "Real": real_prices.ravel(),
    "Predicted": predicted_prices.ravel()
}, index = df.index[-len(real_prices): ]) 
stocks.head()

,Real,Predicted
2019-02-20,3924.239990,4248.814941
2019-02-21,3974.050049,4270.220703
2019-02-22,3937.040039,4296.877441
2019-02-23,3983.530029,4322.457520
2019-02-24,4149.089844,4345.416504


In [21]:
# Plot the real vs predicted values as a line chart
# YOUR CODE HERE!

:NdOverlay   [Variable]
   :Curve   [index]   (value)